Laboration för kursen DD1334
Använder SQL-lite för att kommunicera med en databas som sparar data från vårt projekt i kartgenerering

Först måste vi hämta hem map klasser från github och installera dependencies

In [64]:
! pip install depend #fixa sen
%pip install ipywidgets
! git clone git@github.com:poskusen/dd1334.git

Defaulting to user installation because normal site-packages is not writeable


ERROR: Invalid requirement: '#fixa': Expected package name at the start of dependency specifier
    #fixa
    ^


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Cloning into 'dd1334'...
Host key verification failed.
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.


Vi börjar med att generera en karta som ska sparas.

In [65]:
from Classes.map import Map
import ipywidgets as widgets
from IPython.display import display, clear_output
from draw_PNG import draw_map
ny_karta = Map(5, 50, 0, 0 ,100, realm_name= 'test_karta') # Skapa en karta med bara byar

draw_map(ny_karta)
output = widgets.Output()
with output: # Visa upp kartan
    image = open("test_image.png", "rb").read() 
    
    image_widget = widgets.Image(
        value=image,
        format='png',
        width=1000,            
        height=1000
    )
    display(image_widget)
display(output)


Output()

Sen connectar vi till databasen och skapar ett nytt table för kartor, continenter och allt content på kontinenterna.

Här uppstår ett stort problem som vi undviker genom att använda pickle.
För att spara vår representation av kontinenten så måste vi spara en lista med punkter. Detta blir väldigt tricky då SQL databaser är baserade på att man sparar 1 värde och inte en lista med värden. Något vi undviker när vi bara serialiserar hela objektet. 
En lösning hade varit skapa ett table per lista och därmed en per kontinent, men det är väldigt bökigt och förstör hela poängen med databasen. Och eftersom vi aldrig vill ha enbart en eller flera punkter från kontinent-listan finns det ingen poäng med att skapa ett helt nytt table, då vi aldrig hämtar enskilda värden från listan. Därför kan man se listan snarare som ett objekt, än en lista. Därför serialiserar vi listan och sparar den sen, allt annat hade varit galenskap.

För att ändå visa att det går gör vi det för städerna.
Vi sparar även bara städerna för floder och berg hade inneburit dubbelarbete, och visar egentligen ingenting.

Den enda fördelen med att köra sql är att det går att hämta kontinenter med rätt 'size' från databasen, dvs att jämföra storleken på serilizationen av points_list.

In [66]:
import sqlite3, csv

conn = sqlite3.connect('database.db')
cursor = conn.cursor()
cursor.execute('PRAGMA foreign_keys = OFF;')

taboles = ['maps', 'continents', 'villages']
for table in taboles:
    cursor.execute(f"DROP TABLE IF EXISTS {table}") #Reset or drop all before filled out tables


cursor.execute('PRAGMA foreign_keys = ON;')

cursor.execute('''CREATE TABLE maps(
               id INTEGER PRIMARY KEY AUTOINCREMENT,
               map_size_x INT DEFAULT 1000,
               map_size_y INT DEFAULT 1000,
               name VARCHAR(255) NOT NULL,
               amount_continents INTEGER NOT NULL
               );''') # Skapa ett table för kartor

cursor.execute('''CREATE TABLE continents(
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                points_serialised VARCHAR(65535),
                id_map INTEGER NOT NULL,
                FOREIGN KEY (id_map) REFERENCES maps(id)
               );''') # Skapa ett table för kontintenter


cursor.execute('''CREATE TABLE villages(
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                x INT NOT NULL,
                y INT NOT NULL,
                id_cont INTEGER NOT NULL,
                FOREIGN KEY (id_cont) REFERENCES continents(id)
               );''') # Skapa ett table för byar

conn.commit()

Stäng databas om det behövs

In [5]:
conn.close()

Skapa en enkel serialiserings algoritm som kan spara listor och en som kan återskapa innehållet och göra det till en lista

In [67]:
def stringify(continent_list):
    ser = ''
    for point in continent_list:
        str_point = str(point)
        str_point = str_point.strip('()')
        str_point = str_point.replace(' ','')
        ser +=  str_point + ','
    return ser

    

def unstringify(ser_cont):
    continent_list = []
    lista_nummer = ser_cont.split(',')  # Split the serialized string by commas
    print(lista_nummer)  # For debugging, you can remove this in production

    # Iterate through pairs of numbers (assuming each pair represents x, y coordinates)
    for i in range(0, len(lista_nummer) - 1, 2):
        # Convert the string representation of the numbers to floats
        continent_list.append((float(lista_nummer[i]), float(lista_nummer[i + 1])))

    return continent_list


lista = [(5,5), (6,6), (7,7)] # Testa funktionerna
string_lista = stringify(lista)
print(string_lista)
print(unstringify(string_lista))


5,5,6,6,7,7,
['5', '5', '6', '6', '7', '7', '']
[(5.0, 5.0), (6.0, 6.0), (7.0, 7.0)]


Skapa funktion som extraherar kontinenter, karta och byar och serialisera kontinent_listorna samt lägger till dem och kartan i databasen

In [68]:
def save(map, cursor):
    amount_continents = map.get_continent_amount()
    name_map = map.get_realm_name()
    (x_size, y_size) = map.get_mapsize()
    cursor.execute('''INSERT INTO maps (map_size_x, map_size_y, name, amount_continents)
                    VALUES(?,?,?,?);
                   ''', (x_size, y_size, name_map, amount_continents))
    continents = map.get_continents()
    map_id = cursor.lastrowid
    for cont in continents:
        points_list = cont.get_point_list()
        ser_points = stringify(points_list)
        cursor.execute('''INSERT INTO continents (points_serialised, id_map)
                    VALUES(?,?);
                    ''', (ser_points, map_id))
        villages = cont.get_villages().get_cities_list()
        cont_id = cursor.lastrowid
        for village in villages:
            cursor.execute('''INSERT INTO villages (x, y, id_cont)
                        VALUES(?,?,?);
                           ''', (village[0], village[1], cont_id))

Testa att spara ner en karta

In [69]:
save(ny_karta, cursor)
cursor = conn.execute("SELECT * FROM CONTINENTS")
rows = cursor.fetchall()
for row in rows:
    print(row)

conn.commit() # Commita till databasen

(1, '513.6470611508354,514.8278059758392,463.23869380761,529.8819356809894,439.10668577050575,538.8326625510799,390.351295510237,548.4187087219318,381.2636923116478,553.8080747755027,342.9490103457148,558.5795612350513,310.88719997139174,539.1709495823525,293.271958908861,514.3729811417683,283.4788763120941,468.83340821868603,283.922356106854,462.77623213933765,320.5395943664234,452.93383672450415,299.82900816658184,437.4208086670384,299.7355225705832,437.1074230079383,301.2100507680415,417.46669639368014,299.8080640457309,415.1725615464004,284.89309111274946,365.9341974975957,284.1748880771394,340.48829124836726,299.443847362879,301.0626607461567,344.1648741651458,282.58048871989837,382.8975452212656,273.46437491365447,418.04887123010815,281.07754331661187,424.3473083040689,282.9529297727459,431.67445578967556,300.5038107709579,470.4509125347339,307.80195128518716,476.47202865722426,314.8496305958564,485.6495346298985,329.30311376803104,485.5246169290433,375.4891788494219,466.47496075

Skapa en funktion som hämtar upp en karta och återskapar kartan

In [19]:
def load(karta_id, cursor): 
    cursor.execute('''SELECT * FROM maps
                JOIN continents ON maps.id = continents.id_map
                JOIN villages ON continents.id = villages.id_cont
                WHERE maps.id = ?
                ;
                ''', (karta_id, )) #Hämta alla kartan med karta_id
    
    data = cursor.fetchall()
    for data in data:
        print(data)
        
    amount_continents = data[4]
    list_continents = []
    for i in range(0, amount_continents): #Hämta kontinenterna
        list_continents.append(unstringify(data[6 + i])) #Avserilisera kontinenterna
    for i in range(6 + amount_continents, len(data)): #Hämta kontinenter
        pass
    
    #return Map(amount_continents, 100, 0, 0, 50, (data[1], data[2]), data[3], create_new=False, ) #Återskapa kartan

laddad_karta = load(1, cursor)




(1, 1000, 1000, 'test_karta', 5, 1, '503.2719763602699,419.74282031894523,508.26032140037466,367.3189442841454,524.963378338599,328.7634518188166,550.0510416873522,294.941644129269,609.8487636924392,250.20916408900655,578.5033826858675,261.4158563907907,559.9971611472212,261.389913142466,531.603970315315,267.9479986790609,515.1672390086798,260.67321445741123,491.9129272298425,237.62877820266,480.7906649738416,227.9362055908046,454.5846852299625,200.0293326937068,437.8099513819101,157.78584426495587,441.7426962621755,132.38745357619092,420.9446114243177,118.11063297158199,419.79403940343906,126.13341251039648,401.83274199879594,145.55190018517393,354.8995113953747,149.94510705546287,388.5992941096101,154.0705929891113,416.06129443336044,176.9483219627375,419.66733059451155,181.0667029972551,436.111603145097,200.06869131101712,437.74773606239535,208.97091241539374,438.46604353147615,211.61682320896045,432.33403847615676,219.42787869732945,412.09115784266584,228.31351533374368,409.7213464

AttributeError: 'int' object has no attribute 'split'

In [70]:
from Classes.continent import Continent

#Olle load
def load(karta_id, cursor): 
    continent_cords_list = [] #Every position is a list with all the continents positions
    villages_lists = []

    cursor.execute('''SELECT COUNT(*) 
                      FROM continents 
                      WHERE id_map = ?;''', (karta_id,))
    number_of_continents = cursor.fetchone()[0]  # Count of continents
    
    cursor.execute('''SELECT * 
                  FROM continents 
                  WHERE id_map = ?;''', (karta_id,))#Get all continents
    
    continents = cursor.fetchall()

    for row in continents:
        continent_cords_list.append(unstringify(row[1]))  #Now we have all continent cordinates 

    for continent_id in range (1, number_of_continents+1):
        temp_list = []

        cursor.execute('''SELECT * 
                      FROM villages 
                      WHERE id_cont = ?;''', (continent_id,))
        villages = cursor.fetchall()
        for rows in villages:
            x = rows[1]
            y = rows[2]
            xy = (x,y)
            temp_list.append(xy)
        villages_lists.append(temp_list)

    #Now make continent list containing actual continent instances
    continent_list = []
    for i in range (number_of_continents):
        print("POINTLIST")
        print(continent_cords_list[i])

        temp_cont = Continent((1000,1000), 0, 0, 0 , 0, point_list = continent_cords_list[i], content = [None, villages_lists[i], None, None], LoadMapp=True)
  
        #print(villages_lists[i])
    
        #continent_list.append(temp_cont)

     




load(1, cursor)


['513.6470611508354', '514.8278059758392', '463.23869380761', '529.8819356809894', '439.10668577050575', '538.8326625510799', '390.351295510237', '548.4187087219318', '381.2636923116478', '553.8080747755027', '342.9490103457148', '558.5795612350513', '310.88719997139174', '539.1709495823525', '293.271958908861', '514.3729811417683', '283.4788763120941', '468.83340821868603', '283.922356106854', '462.77623213933765', '320.5395943664234', '452.93383672450415', '299.82900816658184', '437.4208086670384', '299.7355225705832', '437.1074230079383', '301.2100507680415', '417.46669639368014', '299.8080640457309', '415.1725615464004', '284.89309111274946', '365.9341974975957', '284.1748880771394', '340.48829124836726', '299.443847362879', '301.0626607461567', '344.1648741651458', '282.58048871989837', '382.8975452212656', '273.46437491365447', '418.04887123010815', '281.07754331661187', '424.3473083040689', '282.9529297727459', '431.67445578967556', '300.5038107709579', '470.4509125347339', '307

AttributeError: 'Continent' object has no attribute 'point_list'

Testa att spara och hämta en karta

In [ ]:
laddad_karta = load(1, cursor)

draw_map(ny_karta)
output = widgets.Output()
with output: # Visa upp kartan
    image = open("test_image.png", "rb").read() 
    
    image_widget = widgets.Image(
        value=image,
        format='png',
        width=1000,            
        height=1000
    )
    display(image_widget)
display(output)

Testa att spara och hämta flera kartor